In [2]:
# load data
import pandas as pd

# 读取 .pkl 文件
file_path = "./data/df_combined_filtered_grouped.pkl"  # 请根据实际文件路径替换
df = pd.read_pickle(file_path)
# 筛选出 toxicity 列不为空的行
df_non_null_toxicity = df[df['toxicity'].notna()]

# 如果需要知道筛选后的行数
print(f"\n筛选后数据行数: {len(df_non_null_toxicity)}")


筛选后数据行数: 19546
